# LLM with tool

### Introduction

"You don't need to know how a car works to drive it." In this lesson, we'll just focus on how to use LLMs with a single tool.

### Setup

Setup begins with importing the packages ("tools") we need to initialize a model. We'll start with Anthropic and OpenAI models. We'll use LangChain, which provide model agnostic tools for playing with these models.

In [ ]:
# Load environment variables from .env file
from dotenv import load_dotenv
import os

# Get model agnostic tool for initializing models
from langchain.chat_models import init_chat_model

# Quick start for LangGraph
from langgraph.prebuilt import create_react_agent

In [20]:
# Load .env file (this will automatically set all environment variables)
load_dotenv()

# Verify LangSmith is configured
print(f"LangSmith API Key: {'✓ Set' if os.environ.get('LANGSMITH_API_KEY') else '✗ Not set'}")
print(f"Tracing enabled: {'✓ Yes' if os.environ.get('LANGSMITH_TRACING') == 'true' else '✗ No'}")
print(f"OpenAI API Key: {'✓ Set' if os.environ.get('OPENAI_API_KEY') else '✗ Not set'}")


LangSmith API Key: ✓ Set
Tracing enabled: ✓ Yes
OpenAI API Key: ✓ Set


### Intiialize models

Below we use LangChain's init_chat_model() function, which just needs the model name and provider. In this case, we just pass both into the "model" argument.

Once "initialized", we can "invoke" the models by passing it input text to generate output text.

In [4]:
# We are going to be asking a simple math question
question = "What is 2*5?"

In [ ]:
# Initialize models
claude_sonnet = init_chat_model(
    model="anthropic:claude-3-5-sonnet-latest",
    temperature=0
)

In [ ]:

# Invoke the models 
response_claude = claude_sonnet.invoke(question)

# In LangChain, we get a lot of output (not just the response). Let's iterate over the response to see.
print("Claude response:\n")
for output in response_claude:
    print(output)


Claude response:

('content', '2*5 = 10')
('additional_kwargs', {})
('response_metadata', {'id': 'msg_01NTawVyjnzg1L6PU9g33ULD', 'model': 'claude-3-5-sonnet-20241022', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 14, 'output_tokens': 11, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-5-sonnet-20241022'})
('type', 'ai')
('name', None)
('id', 'run--780975d6-0c7c-4ad2-af62-46c2326a6044-0')
('example', False)
('tool_calls', [])
('invalid_tool_calls', [])
('usage_metadata', {'input_tokens': 14, 'output_tokens': 11, 'total_tokens': 25, 'input_token_details': {'cache_read': 0, 'cache_creation': 0, 'ephemeral_5m_input_tokens': 0, 'ephemeral_1h_input_tokens': 0}})


In [27]:
# Initialize models
o3_mini = init_chat_model(
    model="openai:o3-mini"
)

# Invoke the models 
response_openai = o3_mini.invoke(question)

# In LangChain, we get a lot of output (not just the response). Let's iterate over the response to see.
print("OpenAI response:\n")
for output in response_openai:
    print(output)

OpenAI response:

('content', '2 multiplied by 5 equals 10.')
('additional_kwargs', {'refusal': None})
('response_metadata', {'token_usage': {'completion_tokens': 86, 'prompt_tokens': 13, 'total_tokens': 99, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o3-mini-2025-01-31', 'system_fingerprint': 'fp_6c43dcef8c', 'id': 'chatcmpl-CKvCvwqTJXReG7hRb7bWIwpO0hH04', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None})
('type', 'ai')
('name', None)
('id', 'run--5e5805cf-65ad-497c-b85a-9206f4b702c7-0')
('example', False)
('tool_calls', [])
('invalid_tool_calls', [])
('usage_metadata', {'input_tokens': 13, 'output_tokens': 86, 'total_tokens': 99, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 64}})


### Note

To learn more about this output, you can read more about what these mean in [LangChain's documentation](https://python.langchain.com/api_reference/langchain/chat_models/langchain.chat_models.base.init_chat_model.html#langchain.chat_models.base.init_chat_model).

### Tools
Both models got the answer correct, but this is not always the case with math problems, for which reason, we may want to "bind" the LLM to a math tool. Let's create a multiplication and division tool functions that the LLMs can call.


In [ ]:
def multiply(a, b):
    """
    This tool multiplies two numbers and returns the result.
    """
    return a * b

def divide(a, b):
    """
    This tool divides two numbers and returns the result.
    """
    return a / b


In [7]:

# Now let's bind the tool to the model (using Anthropic only)
claude_sonnet_with_tool = claude_sonnet.bind_tools([multiply, divide])

# Now let's invoke the model with the tool
response_claude_with_tool = claude_sonnet_with_tool.invoke(question)

# In LangChain, we get a lot of output (not just the response). Let's iterate over the response to see.
print("Claude response:\n")
for output in response_claude_with_tool:
    print(output)

Claude response:

('content', [{'text': "I'll help you multiply 2 and 5 using the multiply tool.", 'type': 'text'}, {'id': 'toolu_011fxEj5mjp7pi9k5hzRPHhH', 'input': {'a': 2, 'b': 5}, 'name': 'multiply', 'type': 'tool_use'}])
('additional_kwargs', {})
('response_metadata', {'id': 'msg_013DeUgVxS1HHzhYR5PYq5BY', 'model': 'claude-3-5-sonnet-20241022', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 445, 'output_tokens': 86, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-5-sonnet-20241022'})
('type', 'ai')
('name', None)
('id', 'run--906ccbad-7dd0-4f8f-8e56-3073ccbfe270-0')
('example', False)
('tool_calls', [{'name': 'multiply', 'args': {'a': 2, 'b': 5}, 'id': 'toolu_011fxEj5mjp7pi9k5hzRPHhH', 'type': 'tool_call'}])
('invalid_tool_calls', [])
('usage_metadata', {'input_tokens': 445, 'o

### Observation
Notice how the LLM decides to use the tool, "multiply", and provides the tool with the inputs (2 and 5), which are set to "a" and "b". But we don't get an answer. This is because tool calling requires another LLM call to incorporate the response of the tool call. 

In [8]:
# Let's unpack the response_metadata's "content" more clearly to see the result.
response_claude_with_tool.content

[{'text': "I'll help you multiply 2 and 5 using the multiply tool.",
  'type': 'text'},
 {'id': 'toolu_011fxEj5mjp7pi9k5hzRPHhH',
  'input': {'a': 2, 'b': 5},
  'name': 'multiply',
  'type': 'tool_use'}]

In [9]:
# To get the actual result, we need to execute the tool call
# First, let's see what tool was called. 
# The [0] is because we assume there is only one tool call (0 is the first index)
tool_call = response_claude_with_tool.tool_calls[0]
print(f"# of tool calls: {len(response_claude_with_tool.tool_calls)}")
print(f"Tool called: {tool_call['name']}")
print(f"Arguments: {tool_call['args']}")

# of tool calls: 1
Tool called: multiply
Arguments: {'a': 2, 'b': 5}


In [10]:
# Because "Arguments" contains the inputs to the tool, we can use it to execute the tool
result = multiply(**tool_call['args'])

# The result, 10, is then incorporated into the original response on the subsequent LLM call
print(f"Result: {result}")


Result: 10


### LangGraph

If we are to put this altogether, we would need a chat bot that can keep the LLM calls going until the LLM can confidently end the response with an answer. To do this, we'll use LangGraph, which sets LLMs and tools as nodes and their relationships as edges. 

In our case, we have 1 LLM and 1 tool, so we just have to set up the following nodes:
- START
- LLM
- TOOL
- END

And the following edges:
- START --> LLM
- LLM --> TOOL (conditional)
- TOOL --> LLM
- LLM --> END

In [11]:
# Now all your model calls will be automatically traced in LangSmith!
# You can view the traces at: https://smith.langchain.com

# Let's test it with a simple call
test_response = claude_sonnet.invoke("What is 3 + 4?")

print("Response:", test_response.content)
print("\nThis call has been traced to LangSmith!")
print("Check your LangSmith dashboard to see the trace.")


Response: 3 + 4 = 7

This call has been traced to LangSmith!
Check your LangSmith dashboard to see the trace.


In [ ]:
# We are going to use create_react_agent to use LangGraph's prebuilt agent framework. 
# This saves us the trouble of setting up the nodes and edges manually.
agent = create_react_agent(
    model="anthropic:claude-3-5-sonnet-latest",
    tools=[multiply, divide],
    prompt="You are a helpful assistant"
)

In [ ]:
# Run the agent
agent_response = agent.invoke(
    {"messages": [{"role": "user", "content": question}]}
)

In [15]:
# Here is the full response
agent_response

{'messages': [HumanMessage(content='What is 2*5?', additional_kwargs={}, response_metadata={}, id='0abf7e65-fe69-4902-8642-081cb4d3ae82'),
  AIMessage(content=[{'text': "I'll help you multiply 2 and 5 using the multiply tool.", 'type': 'text'}, {'id': 'toolu_01TRYXEFqMnaJAF6Ki3woHaW', 'input': {'a': 2, 'b': 5}, 'name': 'multiply', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01V7kmPch6bfPydRK2YgMhBY', 'model': 'claude-3-5-sonnet-20241022', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 451, 'output_tokens': 86, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-5-sonnet-20241022'}, id='run--04819116-d885-4d63-b70c-357a83a5010d-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 5}, 'id': 'toolu_01TRYXEFqMnaJAF6Ki3woHaW', 'type': 'tool_call'}], usage_

In [16]:
# Let's take a look at the messages
for message in agent_response['messages']:
    print(message.type, ": ", message.content)

human :  What is 2*5?
ai :  [{'text': "I'll help you multiply 2 and 5 using the multiply tool.", 'type': 'text'}, {'id': 'toolu_01TRYXEFqMnaJAF6Ki3woHaW', 'input': {'a': 2, 'b': 5}, 'name': 'multiply', 'type': 'tool_use'}]
tool :  10
ai :  2 multiplied by 5 equals 10.


# That is it

Now you can create your own chat bot! Check out LangSmith for observability.